# Prosta implementacja systemu eksperckiego

Przykład z **AI for Beginners Curriculum**

Zaimplementujemy prosty system ekspercki do rozpoznawania zwierząt na podstawie cech fizycznych. System może być zilustrowany przez następujące drzewo (jest to drzewo częściowe, łatwo można dodać nowe reguły):

![](https://github.com/microsoft/AI-For-Beginners/blob/main/lessons/2-Symbolic/images/AND-OR-Tree.png?raw=true)

## Nasz własny system ekspercki z wnioskowaniem w tył

Zdefiniujmy prosty język do reprezentacji wiedzy bazujący na regułach. Użyjemy klas pythonowych jako słów kluczowych potrzebnych do zdefiniowania reguł. Powinny być trzy typy klas:
* `Ask` reprezentuje pytanie, które musi byćzadane użytkownikowi.Zawiera zbiór potencjalnych odpowiedzi.
* `If` reprezentuje regułę, jest to jedynie opakowanie przechowujące zawartość reguły
* `AND`/`OR` to klasy reprezentujące gałęzie AND/OR w drzewie. Przechowują jedynie listę argumentów. Dla uproszczenia, cała funkcjonalność jest zdefiniowana w klasie bazowej `Content`

In [ ]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x

class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

W naszym systemie, pamięć robocza będzie zawierać listę **faktów** jako **pary atrybut-wartość**. Baza wiedzy jest zdefiniowana jako wielki słownik który przypisuje akcje (nowe fakty które powinny być wstawione do pamięci roboczej) do warunków, wyrażonych jako operacje AND-OR. Niektóre fakty mogą być także uzyskane poprzez zapytanie użytkownika (`Ask`).
<a id='animal_rules'></a>

In [ ]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

By wykonać wnioskowanie w tył, zdefiniujmy klasę `Knowledgebase`. Będzie ona zawierać:
* pamięć roboaczą `memory` - słownik który mapuje atrybuty z wartościami
* Bazę wiedzy `rules` w powyższym formacie

Dwie główne metody to:
* `get` by uzyskać wartość atrybutu, wykonując wnioskowaniem o ile to konieczne. Np. `get('color')` zwróci wartość koloru (zapyta jeśli będzie to konieczne, i przechowa wartość w pamięci roboczej do późniejszego użycia). Jeśli zapytamy `get('color:blue')`, zapyta o kolor, i zwróci wartość `y`/`n` w zależności od koloru.
* `eval` wykonuje właściwe wnioskowanie, czyli przechodzi drzewo AND/OR, ewaluuje podcele, etc.

In [ ]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}

    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        res = self.eval(self.rules['default'],field=name) # pole nieznalezione, użyj domyślnego
        self.memory[name]=res
        return res

    def eval(self,expr,field=None):
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

Zdefiniujmy teraz zwierzęcą bazę danych i dokonajmy konsultacji. Jej wywołanie spowoduje zadanie pytań, na które należy odpowiadać `y`/`n` przy pytaniach tak-nie, albo konkretną liczbą (0..N) przy pytaniach z wieloma wyborami.

In [ ]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n


## Użycie biblioteki PyKnow do wnioskowania w przód

W następnym przykładzie zaimplementujemy wnioskowanie w przód za pomocą jednej z bibliotek do reprezentacji wiedzy, [PyKnow](https://github.com/buguroo/pyknow/). **PyKnow** jest bibilioteką do tworzenia systemów wnioskujących w przód w języku Python.

In [ ]:
import sys
!{sys.executable} -m pip install git+https://github.com/buguroo/pyknow/

In [ ]:
from pyknow import *

Zdefiniujemy nasz system jako klasę która dziedziczy z klasy `KnowledgeEngine`. Każda reguła jest zdefiniowana jako oddzielna funkcja z adnotacją `@Rule`, która precyzuje kiedy reguła powinna zostać odpalona. Wewnątrz reguły możemy dodać nowe fakty używając funkcji `declare` - dodanie tych faktów zaskutkuje wywołaniem kolejnych reguł przez silnik wnioskujący.

**Zadanie 1**

Wzorując się na regule dotyczącej mięsożerców zdefiniuj wszystkie reguły zawarte [tutaj](#animal_rules)

In [ ]:
class Animals(KnowledgeEngine):
    @Rule(OR(AND(Fact('sharp teeth'), Fact('claws'), Fact('forward looking eyes')), Fact('eats meat')))
    def carnivore(self):
        self.declare(Fact('carnivore'))

    @Rule(AND(Fact('mammal'), Fact('has hooves')))
    def ungulate(self):
        self.declare(Fact('ungulate'))

    @Rule(Fact('mammal'))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('has hooves'))
    def hooves(self):
        self.declare(Fact('has hooves'))

    @Rule(AND(Fact('feathers'), Fact('flies'), Fact('lays eggs')))
    def bird(self):
        self.declare(Fact('bird'))

    @Rule(AND(Fact('color', 'red-brown'), Fact('pattern', 'dark spots')))
    def monkey(self):
        self.declare(Fact('animal', 'monkey'))

    @Rule(AND(Fact('color', 'red-brown'), Fact('pattern', 'dark stripes')))
    def tiger(self):
        self.declare(Fact('animal', 'tiger'))

    @Rule(AND(Fact('ungulate'), Fact('long neck'), Fact('long legs'), Fact('pattern', 'dark spots')))
    def giraffe(self):
        self.declare(Fact('animal', 'giraffe'))

    @Rule(AND(Fact('ungulate'), Fact('pattern', 'dark stripes')))
    def zebra(self):
        self.declare(Fact('animal', 'zebra'))

    @Rule(AND(Fact('bird'), Fact('long neck'), Fact('color', 'black and white'), Fact('cannot fly')))
    def ostrich(self):
        self.declare(Fact('animal', 'ostrich'))

    @Rule(AND(Fact('bird'), Fact('swims'), Fact('color', 'black and white'), Fact('cannot fly')))
    def penguin(self):
        self.declare(Fact('animal', 'penguin'))

    @Rule(AND(Fact('bird'), Fact('flies well')))
    def albatross(self):
        self.declare(Fact('animal', 'albatross'))

    @Rule(Fact(animal=MATCH.a))
    def print_result(self, a):
        print('Animal is {}'.format(a['animal']))

    def factz(self, l):
        for x in l:
            self.declare(x)


Skoro zdefiniowaliśmy bazę wiedzy, możemy zapełnić pamięć roboczą jakimiś startowymi faktami, a następnie wywołać metodę `run()` aby wykonać wnioskowanie. Możemy zauważyć, że nowe wywnioskowane fakty zostają dodane do pamięci, włączywszy w to finalny fakt o zwięrzęciu (jeśli ustawiliśmy wszystkie fakty prawidłowo).

In [ ]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact('color', 'black and white'),
    Fact('cannot fly'),
    Fact('swims'),
    Fact('bird')
])
ex1.run()
